In [ ]:
from dataclasses import dataclass
import itertools
import logging
import random
import math
import numpy as np
import json
import pickle
import time
import torch
import sys
import yaml
import seaborn as sns
import matplotlib.pyplot as plt

from omegaconf import OmegaConf
from pathlib import Path
from torch import nn, Tensor
from torch.nn import functional as F
from typing import List, Optional, Tuple
import os
os.chdir("/Users/guotianyu/GitHub/birth")
from data import DataArgs, Dataset, iterate_batches, make_dataset
from ihead_full_model import ModelArgs, Transformer

%load_ext autoreload
%autoreload 2


In [ ]:
n_layers = 2
n_heads = 2
bos_num = 1
train_steps = 2000
path_local = f"/Users/guotianyu/GitHub/birth/gens/delim/model_L{n_layers}_H{n_heads}_lr3-4_bos{bos_num}_delim005/params.yaml"
# path_server = f"/data/tianyu_guo/birth/gens/delim/model_L{n_layers}_H{n_heads}_lr3-4_bos{bos_num}_delim005/params.yaml"
cfg = OmegaConf.load(path_local)
model = Transformer(cfg.model_args)
model.eval()

state_path_local = f"/Users/guotianyu/GitHub/birth/gens/delim/model_L{n_layers}_H{n_heads}_lr3-4_bos{bos_num}_delim005/state_{train_steps}.pt"
state_path_server = f"/data/tianyu_guo/birth/gens/delim/model_L{n_layers}_H{n_heads}_lr3-4_bos{bos_num}_delim005/state_{train_steps}.pt"
state = torch.load(state_path_local, map_location="cpu")
model.load_state_dict(state["model_state_dict"], strict=False)

In [ ]:
import pickle
import numpy as np

with open("/Users/guotianyu/GitHub/birth/data/meta_bos1_d0.pickle", "rb") as f:
    meta_info = pickle.load(f)

# data_cfg = OmegaConf.structured(meta_info)


ds = make_dataset(cfg, meta_info)
ds = Dataset(cfg.data_args, train_test=None)
ds_test = Dataset(cfg.data_args, train_test=None)
ds_test.idxs = ds.idxs
cfg.model_args.vocab_size = ds.num_tokens
x, outs = ds.gen_batch(rng=np.random.default_rng([42, 27]), batch_size=cfg.optim_args.batch_size)
(x, y, outs) = (x[:,:-1], x[:,1:], outs[:,:-1])
x = torch.from_numpy(x)
y = torch.from_numpy(y)
outs = torch.from_numpy(outs)